In [1]:
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor

def start_stable_driver():
    options = Options()
    options.page_load_strategy = 'eager' 
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("detach", True)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    # Buzdolabı sayfaları ağır olabilir, resimleri kapatmak hızı çok artırır
    prefs = {"profile.managed_default_content_settings.images": 2}
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

# Global Ayarlar
BASE_URL = "https://www.sikayetvar.com/arcelik/buzdolabi"
detail_links = [] # Buzdolabı linkleri buraya dolacak

In [2]:
target_pages = 200 # İhtiyaca göre artırabilirsin
current_page = 1 
refresh_interval = 20 

def collect_fridge_links(start_p, end_p):
    global detail_links
    driver = start_stable_driver()
    print(f"🌐 Tarayıcı başlatıldı: {start_p}-{end_p} arası taranıyor...")
    
    for p in range(start_p, end_p + 1):
        try:
            driver.get(f"{BASE_URL}?page={p}")
            time.sleep(3) # Buzdolabı sayfası yüklenme payı
            
            links = driver.find_elements(By.CSS_SELECTOR, "a.complaint-layer")
            
            if len(links) > 0:
                for l in links:
                    href = l.get_attribute("href")
                    if href and "/arcelik/" in href:
                        if not any(d['url'] == href for d in detail_links):
                            detail_links.append({"baslik": l.get_attribute("title"), "url": href})
                print(f"✅ Sayfa {p} bitti. Havuz: {len(detail_links)} link.")
            else:
                print(f"🛑 Sayfa {p} boş! Manuel kontrol gerekebilir.")
                time.sleep(5)
        except Exception as e:
            print(f"⚠️ Hata (Sayfa {p}): {str(e)[:40]}")
    
    driver.quit()

# Ana Döngü
while current_page <= target_pages:
    next_stop = min(current_page + refresh_interval - 1, target_pages)
    collect_fridge_links(current_page, next_stop)
    current_page = next_stop + 1

🌐 Tarayıcı başlatıldı: 1-20 arası taranıyor...
✅ Sayfa 1 bitti. Havuz: 24 link.
✅ Sayfa 2 bitti. Havuz: 48 link.
✅ Sayfa 3 bitti. Havuz: 71 link.
✅ Sayfa 4 bitti. Havuz: 95 link.
✅ Sayfa 5 bitti. Havuz: 118 link.
✅ Sayfa 6 bitti. Havuz: 142 link.
✅ Sayfa 7 bitti. Havuz: 165 link.
✅ Sayfa 8 bitti. Havuz: 188 link.
✅ Sayfa 9 bitti. Havuz: 211 link.
✅ Sayfa 10 bitti. Havuz: 235 link.
✅ Sayfa 11 bitti. Havuz: 259 link.
✅ Sayfa 12 bitti. Havuz: 283 link.
✅ Sayfa 13 bitti. Havuz: 307 link.
✅ Sayfa 14 bitti. Havuz: 330 link.
✅ Sayfa 15 bitti. Havuz: 352 link.
✅ Sayfa 16 bitti. Havuz: 376 link.
✅ Sayfa 17 bitti. Havuz: 399 link.
✅ Sayfa 18 bitti. Havuz: 421 link.
✅ Sayfa 19 bitti. Havuz: 444 link.
✅ Sayfa 20 bitti. Havuz: 467 link.
🌐 Tarayıcı başlatıldı: 21-40 arası taranıyor...
✅ Sayfa 21 bitti. Havuz: 491 link.
✅ Sayfa 22 bitti. Havuz: 514 link.
✅ Sayfa 23 bitti. Havuz: 538 link.
✅ Sayfa 24 bitti. Havuz: 562 link.
✅ Sayfa 25 bitti. Havuz: 586 link.
✅ Sayfa 26 bitti. Havuz: 610 link.
✅ Sayfa 

In [3]:
session = requests.Session()
adapter = requests.adapters.HTTPAdapter(pool_connections=60, pool_maxsize=60)
session.mount('https://', adapter)

def fetch_fridge_detail(item):
    url = item['url']
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    
    for attempt in range(4): # 4 kez deneme hakkı
        try:
            time.sleep(random.uniform(0.2, 0.5) * (attempt + 1))
            res = session.get(url, headers=headers, timeout=12)
            
            if res.status_code == 200:
                soup = BeautifulSoup(res.text, 'html.parser')
                start_node = soup.find("div", {"data-ga-element": "Complaint_Content_Start"})
                content_parts = []
                
                if start_node:
                    for sibling in start_node.find_next_siblings():
                        if sibling.name == "div" and sibling.get("data-ga-element") == "Complaint_Content_End":
                            break
                        if sibling.name == "p":
                            content_parts.append(sibling.get_text(strip=True))
                
                full_text = " ".join(content_parts)
                if not full_text:
                    desc_div = soup.find("div", class_="complaint-detail-description")
                    full_text = desc_div.get_text(strip=True) if desc_div else ""

                if len(full_text) > 5:
                    return {"baslik": item["baslik"], "url": url, "aciklama": full_text}
            elif res.status_code == 429:
                time.sleep(6)
        except:
            continue
    return {"baslik": item["baslik"], "url": url, "aciklama": "HATA: Alınamadı."}

# Çalıştırma
print(f"🚀 {len(detail_links)} adet buzdolabı şikayeti işleniyor...")
final_fridge_data = []
start_time = time.time()

with ThreadPoolExecutor(max_workers=12) as executor:
    futures = [executor.submit(fetch_fridge_detail, link) for link in detail_links]
    for i, future in enumerate(futures, 1):
        final_fridge_data.append(future.result())
        if i % 100 == 0:
            elapsed = time.time() - start_time
            rem = (elapsed / i) * (len(detail_links) - i) / 60
            print(f"📊 Durum: {i}/{len(detail_links)} | Kalan: ~{rem:.1f} dk")



🚀 2700 adet buzdolabı şikayeti işleniyor...
📊 Durum: 100/2700 | Kalan: ~51.4 dk
📊 Durum: 200/2700 | Kalan: ~43.4 dk
📊 Durum: 300/2700 | Kalan: ~36.7 dk
📊 Durum: 400/2700 | Kalan: ~35.1 dk
📊 Durum: 500/2700 | Kalan: ~31.8 dk
📊 Durum: 600/2700 | Kalan: ~30.4 dk
📊 Durum: 700/2700 | Kalan: ~30.1 dk
📊 Durum: 800/2700 | Kalan: ~27.4 dk
📊 Durum: 900/2700 | Kalan: ~25.2 dk
📊 Durum: 1000/2700 | Kalan: ~24.4 dk
📊 Durum: 1100/2700 | Kalan: ~22.9 dk
📊 Durum: 1200/2700 | Kalan: ~20.2 dk
📊 Durum: 1300/2700 | Kalan: ~19.0 dk
📊 Durum: 1400/2700 | Kalan: ~17.4 dk
📊 Durum: 1500/2700 | Kalan: ~15.7 dk
📊 Durum: 1600/2700 | Kalan: ~14.2 dk
📊 Durum: 1700/2700 | Kalan: ~13.5 dk
📊 Durum: 1800/2700 | Kalan: ~11.6 dk
📊 Durum: 1900/2700 | Kalan: ~10.6 dk
📊 Durum: 2000/2700 | Kalan: ~9.2 dk
📊 Durum: 2100/2700 | Kalan: ~8.2 dk
📊 Durum: 2200/2700 | Kalan: ~6.7 dk
📊 Durum: 2300/2700 | Kalan: ~5.5 dk
📊 Durum: 2400/2700 | Kalan: ~4.1 dk
📊 Durum: 2500/2700 | Kalan: ~2.7 dk
📊 Durum: 2600/2700 | Kalan: ~1.4 dk
📊 Durum: 2

In [4]:
# Kaydet
pd.DataFrame(final_fridge_data).to_excel("arcelik_buzdolabi_final.xlsx", 
                                         index=False)
print("✅ BUZDOLABI İŞLEMİ TAMAMLANDI!")

✅ BUZDOLABI İŞLEMİ TAMAMLANDI!
